In [1]:
import os
# import ollama
# from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_community.utilities import OpenWeatherMapAPIWrapper
from dotenv import load_dotenv

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
TYPHOON_KEY = os.getenv("TYPHOON_KEY")
WEATHER_KEY = os.getenv("WEATHER_KEY")

print("TAVILY_API_KEY:", TAVILY_API_KEY)
print("TYPHOON_KEY:", TYPHOON_KEY)
print("WEATHER_KEY:", WEATHER_KEY)

TAVILY_API_KEY: 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
TYPHOON_KEY: sk-2GMRcYlJ76Wk2jZDzPHV1Rb8oTxZJTimqRuyj7eywhOVbJNM
WEATHER_KEY: b440848f82cf8ce1ef3b330c2b2a1de1
